In [2]:
# start and import h2o
# set seed
import h2o
h2o.init()
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.targetencoder import TargetEncoder
from h2o.estimators.gbm import H2OGradientBoostingEstimator 
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
SEED = 12345

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_152-release"; OpenJDK Runtime Environment (build 1.8.0_152-release-1056-b12); OpenJDK 64-Bit Server VM (build 25.152-b12, mixed mode)
  Starting server from /anaconda3/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/l6/bdpqrx0s7db19zx5zy46_tkr0000gn/T/tmpg4p6xl2n
  JVM stdout: /var/folders/l6/bdpqrx0s7db19zx5zy46_tkr0000gn/T/tmpg4p6xl2n/h2o_netinupur_started_from_python.out
  JVM stderr: /var/folders/l6/bdpqrx0s7db19zx5zy46_tkr0000gn/T/tmpg4p6xl2n/h2o_netinupur_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.4
H2O cluster version age:,2 months and 22 days
H2O cluster name:,H2O_from_python_netinupur_yvgxyk
H2O cluster total nodes:,1
H2O cluster free memory:,1.778 Gb
H2O cluster total cores:,0
H2O cluster allowed cores:,0
H2O cluster status:,"accepting new members, healthy"


In [3]:
col_types = {'hotel_cluster': 'enum',
             'user_id': 'enum',
             'srch_destination_id': 'enum',
             'hotel_location_id': 'enum',
             'srch_ci':'enum',
             'srch_co':'enum',
             'date_time':'enum',
             'user_location_id': 'enum',
             'srch_destination_type_id': 'enum',
             'channel': 'enum',
             'is_package': 'enum',
             'is_booking': 'enum'
            }


train= h2o.import_file(path="train.csv",col_types=col_types)
test= h2o.import_file(path="test.csv",col_types=col_types)
valid= h2o.import_file(path="valid.csv",col_types=col_types)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
train.describe()

Rows:562167
Cols:21




,hotel_cluster,user_id,srch_destination_id,hotel_location_id,srch_ci,srch_co,date_time,user_location_id,srch_destination_type_id,channel,is_package,is_booking,srch_ci_month,srch_ci_dayofweek,srch_dayofweek,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,length_stay,book_advance_day,cnt
type,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,int,int,int,int,int,int,int,int,int
mins,,,,,,,,,,,,,1.0,1.0,1.0,0.0,0.0,1.0,-148.0,0.0,1.0
mean,,,,,,,,,,,,,6.193673765980572,4.294234631346201,3.8751456417754766,1.9533786223666625,0.37705521668827885,1.102131573002329,3.518050686006117,56.35505819445109,4.004384818034502
maxs,,,,,,,,,,,,,12.0,7.0,7.0,9.0,9.0,8.0,366.0,6200.0,281.0
sigma,,,,,,,,,,,,,3.353108715152266,2.006741954865364,1.917421207914957,0.8347708621573421,0.7254928223355738,0.42211452491383233,3.529290983133775,67.24413540201998,6.092056172936305
zeros,,,,,,,,,,,,,0,0,0,1598,415704,0,5198,16985,0
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,10,725753,8288,882,2013-01-11,2013-01-12,07JAN13:00:02:02,2112,1,5,0,1,1.0,6.0,2.0,2.0,0.0,1.0,1.0,4.0,1.0
1,98,368084,44045,449,2013-03-04,2013-03-06,07JAN13:00:04:58,809,3,3,0,0,3.0,2.0,2.0,4.0,0.0,1.0,2.0,56.0,3.0
2,7,756870,1811,921,2013-01-08,2013-01-09,07JAN13:00:05:56,2112,1,0,0,0,1.0,3.0,2.0,2.0,1.0,1.0,1.0,1.0,6.0


In [5]:
#assign target and inputs
y = 'is_booking'
print(y)

is_booking


In [12]:
# train target encoder for hotel_location_id and user_location_id
e_columns = ['hotel_location_id','user_location_id']
te_ = h2o.targetencoder.TargetEncoder(x=e_columns, y=y)
train[y] = train[y].asfactor()
_ = te_.fit(train)
e_train = te_.transform(frame=train, holdout_type='loo', seed=12345)
e_valid = te_.transform(frame=valid, holdout_type='loo', seed=12345)
e_test = te_.transform(frame=test, holdout_type='loo', seed=12345)

In [13]:
#train encode for srch_Destination_id 
e_columns_2=['srch_destination_id']
te_ = h2o.targetencoder.TargetEncoder(x=e_columns_2, y=y)
e_train[y] = e_train[y].asfactor()
_ = te_.fit(e_train)
e_train = te_.transform(frame=e_train, holdout_type='loo', seed=12345)
e_valid = te_.transform(frame=e_valid, holdout_type='loo', seed=12345)
e_test = te_.transform(frame=e_test, holdout_type='loo', seed=12345)

In [14]:
e_train.describe()

Rows:562167
Cols:24




,srch_destination_id,user_location_id,hotel_location_id,hotel_cluster,user_id,srch_ci,srch_co,date_time,srch_destination_type_id,channel,is_package,is_booking,srch_ci_month,srch_ci_dayofweek,srch_dayofweek,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,length_stay,book_advance_day,cnt,hotel_location_id_te,user_location_id_te,srch_destination_id_te
type,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,int,int,int,int,int,int,int,int,int,real,real,real
mins,,,,,,,,,,,,,1.0,1.0,1.0,0.0,0.0,1.0,-148.0,0.0,1.0,-0.009986411333084106,-0.009840749082234871,-0.009996926784515381
mean,,,,,,,,,,,,,6.1936737659805745,4.294234631346207,3.8751456417754873,1.953378622366664,0.3770552166882803,1.102131573002328,3.5180506860061116,56.35505819445118,4.004384818034503,0.20375859499151272,0.20369744585536592,0.20237693510988505
maxs,,,,,,,,,,,,,12.0,7.0,7.0,9.0,9.0,8.0,366.0,6200.0,281.0,0.9943375872391644,0.790806913049144,1.0001233952609774
sigma,,,,,,,,,,,,,3.3531087151522674,2.0067419548653596,1.9174212079149617,0.8347708621573414,0.725492822335575,0.4221145249138319,3.5292909831337727,67.24413540201995,6.092056172936318,0.09321756988627682,0.031066344655911085,0.10860649549146467
zeros,,,,,,,,,,,,,0,0,0,1598,415704,0,5198,16985,0,0,0,0
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,10,779,397,7,562657,2013-10-09,2013-10-13,04MAR13:20:41:27,3,9,0,0,10.0,4.0,2.0,2.0,0.0,1.0,4.0,219.0,1.0,0.39142249740087065,0.20005846380452455,0.2927211096774656
1,10,805,397,28,761376,2013-03-31,2013-04-01,30MAR13:22:41:22,3,0,0,1,3.0,1.0,7.0,2.0,3.0,1.0,1.0,1.0,3.0,0.3939858382481795,0.22015671516008933,0.1785091968907301
2,100,1363,1484,67,538688,2013-03-03,2013-03-05,27JAN13:15:36:41,1,9,1,0,3.0,1.0,1.0,1.0,0.0,1.0,2.0,35.0,1.0,0.24909711122448577,0.17352099830454046,0.25291611388668006


In [15]:
e_valid.describe()

Rows:421626
Cols:24




,srch_destination_id,user_location_id,hotel_location_id,hotel_cluster,user_id,srch_ci,srch_co,date_time,srch_destination_type_id,channel,is_package,is_booking,srch_ci_month,srch_ci_dayofweek,srch_dayofweek,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,length_stay,book_advance_day,cnt,hotel_location_id_te,user_location_id_te,srch_destination_id_te
type,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,int,int,int,int,int,int,int,int,int,real,real,real
mins,,,,,,,,,,,,,1.0,1.0,1.0,0.0,0.0,0.0,-6.0,0.0,1.0,-0.1560486080029326,-0.16133638721145477,-0.16172013979591215
mean,,,,,,,,,,,,,7.024806344959747,4.299194072471816,3.8737459264846135,2.002959969261861,0.32349048682955983,1.1051619207544117,3.356045405169503,53.91042060973479,3.9013723062619454,0.20666097973468867,0.2039072916578801,0.20385504677324998
maxs,,,,,,,,,,,,,12.0,7.0,7.0,9.0,9.0,8.0,366.0,586.0,307.0,0.9926566253918592,0.7524791121691421,1.0817684223806303
sigma,,,,,,,,,,,,,2.1785693390632743,2.012698062848197,1.879432102984028,0.8570612244091304,0.7357806900732015,0.4301813284224013,3.221676205174172,66.31496066594421,5.925942535883332,0.09532471577808616,0.031944166643194015,0.1111626030365323
zeros,,,,,,,,,,,,,0,0,0,751,339148,4,3395,13716,0,0,0,0
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,39393,1467,1200,30,120030,2014-09-05,2014-09-06,29JUL14:16:10:06,4,5,0,0,9.0,6.0,3.0,2.0,0.0,1.0,1.0,38.0,1.0,0.20773286292518395,0.2037437273977306,0.20374372739773083
1,56360,1815,1230,61,27007,2014-06-07,2014-06-08,03JUN14:14:37:33,3,5,0,0,6.0,7.0,3.0,3.0,0.0,1.0,1.0,4.0,2.0,0.27380023290306854,0.2037437273977306,0.20374372739773083
2,57923,1085,1268,82,799567,2014-07-26,2014-07-27,03JUL14:09:27:35,4,2,0,1,7.0,7.0,5.0,2.0,0.0,3.0,1.0,23.0,3.0,0.15841573902719536,0.2037437273977306,0.20374372739773083


In [16]:
e_test.describe()

Rows:421626
Cols:24




,srch_destination_id,user_location_id,hotel_location_id,hotel_cluster,user_id,srch_ci,srch_co,date_time,srch_destination_type_id,channel,is_package,is_booking,srch_ci_month,srch_ci_dayofweek,srch_dayofweek,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,length_stay,book_advance_day,cnt,hotel_location_id_te,user_location_id_te,srch_destination_id_te
type,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,int,int,int,int,int,int,int,int,int,real,real,real
mins,,,,,,,,,,,,,1.0,1.0,1.0,0.0,0.0,0.0,-335.0,0.0,1.0,-0.16059111099876253,-0.1603938661161789,-0.16118653636611785
mean,,,,,,,,,,,,,8.246768463045447,4.319273479339526,3.815213008685426,1.985895082371581,0.24743967402389802,1.1008974778595242,3.4177754692547344,58.73650106966826,3.968185073975507,0.20246958568256485,0.20417833795674628,0.20032527719129295
maxs,,,,,,,,,,,,,12.0,7.0,7.0,9.0,9.0,8.0,370.0,696.0,341.0,0.9955166970509473,0.7553484690397932,1.0764339854871672
sigma,,,,,,,,,,,,,3.968762468833243,2.0053597437845294,1.8833998317156635,0.8465260000341048,0.6592710150455872,0.4286970182522297,4.372739296296807,71.50235852710206,6.277147524833237,0.09380931837699444,0.03375852427589929,0.10884030068797207
zeros,,,,,,,,,,,,,0,0,0,624,358422,13,3106,14977,0,0,0,0
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,536,1973,1626,20,668807,2014-09-09,2014-09-16,06SEP14:03:50:01,3,9,0,1,9.0,3.0,7.0,1.0,0.0,1.0,7.0,3.0,11.0,0.20374372739773058,0.2037437273977306,0.20374372739773083
1,64591,226,1269,58,373106,2014-12-25,2014-12-30,26OCT14:19:52:31,1,5,0,0,12.0,5.0,1.0,2.0,0.0,1.0,5.0,60.0,1.0,0.08262084866747443,0.2037437273977306,0.20374372739773083
2,33460,468,1597,20,193647,2014-12-04,2014-12-07,29NOV14:17:28:33,6,2,0,0,12.0,5.0,7.0,2.0,0.0,1.0,3.0,5.0,2.0,0.048618468174445845,0.2037437273977306,0.20374372739773083


In [17]:
e_train_as_pandas=e_train.as_data_frame()
e_valid_as_pandas=e_valid.as_data_frame()
e_test_as_pandas=e_test.as_data_frame()

In [18]:
e_valid_as_pandas.to_csv('valid_encoded.csv',index=False)

In [19]:
e_test_as_pandas.to_csv('test_encoded.csv',index=False)

In [20]:
e_train_as_pandas.to_csv('train_encoded.csv',index=False)